This notebook simply allows to visualize particular time windows in the data

In [ ]:
import datetime
import math

import yaml
from matplotlib import pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 15})  # font size of matplotlib figures

from src.utils.data_reading.features_extractor import STFTFeaturesExtractor
from src.utils.data_reading.sound_file_manager import WavFilesManager

In [ ]:
root_dir = "PATH_TO_DATASET"  # path of the dataset where we expect to find the yaml of the data
events_yaml_path = "../../../data/events.yaml"  # file listing interesting events
chosen_events = ["T-wave_4", "H-wave_1", "Cryogenic_1", "Uncertain_1", "Ship-noise_1", "Antarctic-blue-whale-Z-calls_1"]  # choice of the events to show (see events.yaml)
features_extractor = STFTFeaturesExtractor(None, vmin=60, vmax=140, axis_labels=False)  # spectrograms generator

In [ ]:
with open(events_yaml_path, "r") as f:
    available_events = yaml.load(f, Loader=yaml.BaseLoader)

with open(f"{root_dir}/datasets.yaml", "r") as f:
    all_params = yaml.load(f, Loader=yaml.BaseLoader)

In [ ]:
nr = math.ceil(len(chosen_events)/2)
ratio = 0.7
plt.subplots(nr, 2, figsize=(18*ratio,nr*5*ratio))

for i, chosen_event in enumerate(chosen_events):
    event = available_events[chosen_event]
    output_path = f'../../../data/figures/{event}.png'
    params = all_params[event["dataset"]]

    manager = WavFilesManager(f'{root_dir}/{params["root_dir"]}/{event["station"]}')
    features_extractor.manager = manager

    segment_duration_s = int(event["segment_duration_s"])
    segment_duration = datetime.timedelta(seconds=segment_duration_s) # duration of the segment around the event
    segment_center = datetime.datetime.strptime(event["segment_center"], "%Y%m%d_%H%M%S") # event to show

    f, t, features = features_extractor.get_features(segment_center - segment_duration / 2, segment_center + segment_duration / 2)
    features = 255*(features - features_extractor.vmin) / (features_extractor.vmax - features_extractor.vmin)

    ax = plt.subplot(nr, 2, i+1)
    ax.imshow(features, extent=(0, segment_duration_s, f[-1], f[0]) ,aspect="auto", cmap="inferno", vmin=0, vmax=255)
    
    ax.text(0.015, 0.90, f"{chr(97+i)})", transform=ax.transAxes, size=20, weight='bold', ha="left", color='white')
    ax.text(0.98, 0.9, chosen_event.split("_")[0].replace("-"," "), transform=ax.transAxes, size=20, weight='bold', ha="right", color='white')

    if "arrow_offset" in event:
        ax.arrow(int(event["arrow_offset"])+segment_duration_s/2, 112, 0, -5,
                                    length_includes_head=True, head_width=5, head_length=13, color="white")
    
    if i >= len(chosen_events)-2:
        # last row
        ax.set_xlabel("Time (s)")
        
    if i % 2 == 0:
        # first column
        ax.set_ylabel("Frequency (Hz)")
        
        

plt.tight_layout()
plt.savefig("../../../data/figures/example_spectrograms_diversity.png", dpi=200)